<a href="https://colab.research.google.com/github/nic-fp/docs/blob/master/ResNetFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K


class Resnet:
  @staticmethod
  def residual_model(data, K, stride, chanDim, red=False, reg = .0001, bnEps = 2e-5, bnMom = .9):
    shortcut = data
    
    #first block
    bn1 = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(data)
    act1 = Activation('relu')(bn1)
    conv1 = Conv2D(int(K*.25), (1,1), use_bias = False, kernel_regularizer=l2(reg))(act1)

    #second block
    bn2 = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(conv1)
    act2 = Activation('relu')(bn2)
    conv2 = Conv2D(int(K*.25), (3,3),strides=stride, padding= 'same',use_bias = False, kernel_regularizer=l2(reg))(act2)

    #third block
    bn3 = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(conv2)
    act3 = Activation('relu')(bn3)
    conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

    if red:
      shortcut = Conv2D(K, (1,1), strides= stride, use_bias= False, kernel_regularizer= l2(reg))(act1)
    
    x = add([conv3,shortcut]) #check if order matters
    return x

  @staticmethod

  def build(width, height, depth, classes, stages, filters, reg= .0001, bnEps = 2e-5, bnMom = .9):
    inputShape = (height, width, depth)
    chanDim = -1

    #update if using Channels 1st data

    if K.image_data_format() == 'channels_first':
      inputShape = (depth, height, width)
      chanDim = 1
    

    inputs = Input(shape = inputShape)
    x = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(inputs)
    x = Conv2D(filters[0], (5,5),use_bias = False, kernel_regularizer=l2(reg))(x)
    x = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D((1,1))(x)
    x = MaxPooling2D((3,3), strides = (2,2))(x)

    for i in range(0,len(stages)):  # This is the resnet part. For each number in the list of number of stages = the number of residual layers on each other
      stride = (1, 1) if i == 0 else (2, 2)
      x = Resnet.residual_model(x,filters[i+1], stride, chanDim=chanDim, red = True, bnEps= bnEps, bnMom = bnMom)

      for layers in range(0,stages[i]-1):
        x = Resnet.residual_model(x,filters[i+1],stride = (1,1) ,chanDim=chanDim, red = False, bnEps= bnEps, bnMom = bnMom)

    x = BatchNormalization(axis= chanDim, epsilon = bnEps, momentum = bnMom)(x)
    x = Activation('relu')(x)
    x = AveragePooling2D((2,2))(x)
    x = Flatten()(x)
    x = Dense(classes, kernel_regularizer = l2(reg))(x)
    x = Activation('softmax')(x)

    model = Model(inputs,x, name = 'Resnet')

    return model

Using TensorFlow backend.


In [0]:
import numpy as np
np.random.seed(1337) # for reproducibility
from keras.models import Sequential
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout,BatchNormalization

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((60000,28,28,1))
x_test = x_test.reshape((10000,28,28,1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


#98% val accuracy on MNIST after 1 epoch then starts slightly overfitting long term about 64s per epoch
model = Resnet.build(width = 28,height = 28, depth = 1,classes = 10,stages = (3,4,6),filters = (64, 128, 256, 512))
#model.add(Resnet)

model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size= 80, epochs=2,validation_split=.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 83s 2ms/step - loss: 0.6365 - acc: 0.9571 - val_loss: 0.4963 - val_acc: 0.9853
Epoch 2/2
48000/48000 [==============================] - 64s 1ms/step - loss: 0.4312 - acc: 0.9854 - val_loss: 0.3984 - val_acc: 0.9788


In [2]:
import numpy as np
np.random.seed(1337) # for reproducibility
from keras.models import Sequential
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout,BatchNormalization

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape((50000,32,32,3))
x_test = x_test.reshape((10000,32,32,3))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


#73% val acc after 4 epoch  starting to overfit 28s per epoch  might try w dif adam optimizer like last resnet
model = Resnet.build(width = 32,height = 32, depth = 3,classes = 10,stages = (3,4,6),filters = (64, 128, 256, 512))
#model.add(Resnet)

model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size= 80, epochs=4,validation_split=.2)
#model.evaluate(x_test,y_test)

170500096/170498071 [==============================] - 6s 0us/step















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 41s 1ms/step - loss: 1.9486 - acc: 0.4884 - val_loss: 1.6717 - val_acc: 0.5879
Epoch 2/4
40000/40000 [==============================] - 28s 704us/step - loss: 1.4572 - acc: 0.6607 - val_loss: 1.3788 - val_acc: 0.6796
Epoch 3/4
40000/40000 [==============================] - 28s 703us/step - loss: 1.2091 - acc: 0.7355 - val_loss: 1.2321 - val_acc: 0.7243
Epoch 4/4
40000/40000 [==============================] - 28s 695us/step - loss: 1.0591 - acc: 0.7803 - val_loss: 1.1623 - val_acc: 0.7420


In [3]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 3s 297us/step


[1.1741556305885315, 0.7382]

In [0]:
import numpy as np
from keras.models import Sequential
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout,BatchNormalization

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape((50000,32,32,3))
x_test = x_test.reshape((10000,32,32,3))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# 71% val accuracy after 4 epochs 156s per epoch
model = Resnet.build(width = 32,height = 32, depth = 3,classes = 10,stages = (6,10,16),filters = (64, 128, 256, 512))
#model.add(Resnet)

model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size= 80, epochs=4,validation_split=.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 197s 5ms/step - loss: 2.9483 - acc: 0.3893 - val_loss: 2.7068 - val_acc: 0.4143
Epoch 2/4
40000/40000 [==============================] - 157s 4ms/step - loss: 2.1989 - acc: 0.5484 - val_loss: 2.0316 - val_acc: 0.5742
Epoch 3/4
40000/40000 [==============================] - 157s 4ms/step - loss: 1.7217 - acc: 0.6550 - val_loss: 1.5778 - val_acc: 0.6791
Epoch 4/4
40000/40000 [==============================] - 156s 4ms/step - loss: 1.4257 - acc: 0.7184 - val_loss: 1.3732 - val_acc: 0.7178


In [0]:
import numpy as np
from keras.models import Sequential
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D, Dropout,BatchNormalization

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape((50000,32,32,3))
x_test = x_test.reshape((10000,32,32,3))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


#44% val after 4 epochs. need to comment out avg pooling layer to use this 
model = Resnet.build(width = 32,height = 32, depth = 3,classes = 10,stages = (3,4,6,10,16),filters = (16,32,64, 128, 256, 512))
#model.add(Resnet)

model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size= 80, epochs=4,validation_split=.2)

170500096/170498071 [==============================] - 10s 0us/step














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples
Epoch 1/4
40000/40000 [==============================] - 146s 4ms/step - loss: 3.2097 - acc: 0.2252 - val_loss: 2.9660 - val_acc: 0.3074
Epoch 2/4
40000/40000 [==============================] - 105s 3ms/step - loss: 2.7720 - acc: 0.3417 - val_loss: 2.6691 - val_acc: 0.3616
Epoch 3/4
40000/40000 [==============================] - 107s 3ms/step - loss: 2.4674 - acc: 0.4025 - val_loss: 2.3368 - val_acc: 0.4219
Epoch 4/4
40000/40000 [==============================] - 108s 3ms/step - loss: 2.2113 - acc: 0.4477 - val_loss: 2.1517 - val_acc: 0.4475
